# Physiological dataset Colab Demo

- Demo core visulization in the paper (YT.Wei et al., xxx)
- Single session-level cellular (2P) analysis (384 neurons from single optic plane)
- .db & .parquet provide for batch-level analysis

![Workflow](https://github.com/ytsimon2004/rscvp/blob/rev/docs/source/_static/physiological_data_analysis_workflow.png?raw=true)



## Setup virtual environment

In [ ]:
# Install uv (fast Python package manager)
!curl -LsSf https://astral.sh/uv/install.sh | sh

# Install aria2 for faster downloading
!apt-get update -qq
!apt-get install aria2 -qq

# Clone code repo
!git clone https://github.com/ytsimon2004/rscvp.git

# Install using uv
!uv pip install ./rscvp[all]

In [ ]:
import logging
import os
import sqlite3

import polars as pl

from rscvp.model.bayes_decoding.main_decode_analysis import DecodeAnalysisOptions
from rscvp.retinotopic.main_retinotopic_map import RetinotopicMapOptions
from rscvp.selection.main_neuropil_error import NeuropilErrOptions
from rscvp.selection.main_trial_reliability import TrialReliabilityOptions
from rscvp.spatial.main_place_field import PlaceFieldsOptions
from rscvp.spatial.main_position_sort import PositionSortOptions
from rscvp.spatial.main_slb import PositionLowerBoundOptions
from rscvp.statistic.parq.main_topo_metric_gsp import TopoMetricParQ
from rscvp.statistic.parq.main_value_gsp import ValueParQ
from rscvp.statistic.sql.main_base_cls import BaseClassStat
from rscvp.statistic.sql.main_fov import FieldOfViewOptions
from rscvp.util.database import download_database
from rscvp.util.util_demo import mkdir_demo_dataset, run_demo
from rscvp.visual.main_polar import VisualPolarOptions
from rscvp.visual.main_reliability import VisualReliabilityOptions
from rscvp.visual.main_sftf_pref import VisualSFTFPrefOptions
from rscvp.visual.main_tuning import VisualTuningOptions


In [ ]:
# silence non-important warning
logging.getLogger('matplotlib.font_manager').setLevel(logging.ERROR)

### Token Request

In [ ]:
# contact author since paper is not published yet
TOKEN = ...

# download dataset from either zenodo or figshare
mkdir_demo_dataset(token=TOKEN, source='figshare')

## Session-level Demo
- Analysis that used for single recording session

### Retinotopic Phase Map
- See the visual areas + RSC location in dorsal cortex
- Used in `Fig.S5`


In [ ]:
class ExampleRetinotopic(RetinotopicMapOptions):
    exp_date = '210302'
    animal_id = 'YW008'
    debug_mode = True


run_demo(ExampleRetinotopic, token=TOKEN, clean_cached=False)

### Selection pipeline
- Run pre-selection pipeline for neuronal selection

In [ ]:
# exclude the neuropil contamination neuron
class ExampleNeuropilErr(NeuropilErrOptions):
    exp_date = '210315'
    animal_id = 'YW006'
    plane_index = 0


# active neuron in treadmill
class ExampleTrialReliability(TrialReliabilityOptions):
    exp_date = '210315'
    animal_id = 'YW006'
    plane_index = 0
    session = 'light'


# active neuron respond to visual stimuli
class ExampleVisualReliability(VisualReliabilityOptions):
    exp_date = '210315'
    animal_id = 'YW006'
    plane_index = 0


run_demo(ExampleNeuropilErr, token=TOKEN, clean_cached=False)
run_demo(ExampleTrialReliability, token=TOKEN, clean_cached=False)
run_demo(ExampleVisualReliability, token=TOKEN, clean_cached=False)

### Position map
- See position tuning across trials
- Used in `Fig.2A`, `Fig.S1`, `Fig.S2`, `Fig.S4D–S4G`, `Fig.S8`

In [ ]:
# loop through all neurons to get place field
class ExamplePlacefield(PlaceFieldsOptions):
    exp_date = '210315'
    animal_id = 'YW006'
    plane_index = 0
    session = 'light'
    peak_baseline_thres = 0.3
    neuron_id = None


# run all cell use neuron_id = None to select position-tuned (output.csv already cached in demo data for saving time)
class ExamplePositionMap(PositionLowerBoundOptions):
    exp_date = '210315'
    animal_id = 'YW006'
    plane_index = 0
    neuron_id = 0
    session = 'light'
    shuffle_times = 300
    with_heatmap = True
    signal_type = 'spks'
    use_default = True
    debug_mode = True
    do_signal_smooth = True


run_demo(ExamplePlacefield, token=TOKEN, clean_cached=False)
# run_demo(ExamplePositionMap, token=TOKEN, clean_cached=False)

### Trial-averaged deconvolved ∆F/F0 sorted activity

- Example shows only a single session (use indices in odd-sorted and plot with even trials)
- Used in `Fig.2B`, `Fig.S2A–S2B`, `Fig.S3E`, `Fig.S4D`


In [ ]:
class ExampleSortedOdd(PositionSortOptions):
    exp_date = '210315'
    animal_id = 'YW006'
    plane_index = 0
    signal_type = 'spks'
    pc_selection = 'slb'
    used_session = 'light'
    use_trial = 'light-odd'
    debug_mode = True


class ExampleSortedEven(PositionSortOptions):
    exp_date = '210315'
    animal_id = 'YW006'
    plane_index = 0
    signal_type = 'spks'
    pc_selection = 'slb'
    used_session = 'light'
    use_trial = 'light-even'
    use_sorted_strategy = 'light-odd'
    debug_mode = True


run_demo(ExampleSortedOdd, token=TOKEN, clean_cached=False)
run_demo(ExampleSortedEven, token=TOKEN, clean_cached=False)

### Bayes Decoding Analysis
- Decode animal's position with population neural activity
- Used in `Fig.3A–3C`, `Fig.S3A`, `Fig.S4E`

In [ ]:
class ExampleDecoding(DecodeAnalysisOptions):
    exp_date = '210315'
    animal_id = 'YW006'
    plane_index = 0
    session = 'light'
    used_session = 'light'
    random = 200
    pos_bins = 100
    spatial_bin_size = 1.5
    cross_validation = 'odd'
    pre_selection = True
    signal_type = 'df_f'
    use_default = True
    debug_mode = True
    cache_version = 0
    analysis_type = 'overview'  # try `confusion_matrix` to see
    plot_concat_time = True


run_demo(ExampleDecoding, token=TOKEN, clean_cached=False)

### Visual reliability

- For identifying a visual responsive neuron
- Used in `Fig.4A`, `Fig.S6`, `Fig.S8`

In [ ]:
class ExampleReliability(VisualReliabilityOptions):
    exp_date = '210315'
    animal_id = 'YW006'
    plane_index = 0
    neuron_id = 2
    signal_type = 'df_f'
    use_default = True
    debug_mode = True


run_demo(ExampleReliability, token=TOKEN, clean_cached=False)

### Visual Tuning curve
- Calcium raw transients with trial-averaged
- Used in `Fig.5A`,`Fig.5E`, `Fig.5H`, `Fig.S6`


In [ ]:
class ExampleVisualTuning(VisualTuningOptions):
    exp_date = '210315'
    animal_id = 'YW006'
    plane_index = 0
    neuron_id = 2
    signal_type = 'df_f'
    invalid_cache = True
    use_default = True
    debug_mode = True


run_demo(ExampleVisualTuning, token=TOKEN, clean_cached=False)

### Visual Polar plot
- See visual responses in different directions
- Used in `Fig.5B`,`Fig.5E`, `Fig.5H`


In [ ]:
class ExamplePolars(VisualPolarOptions):
    exp_date = '210315'
    animal_id = 'YW006'
    plane_index = 0
    neuron_id = 2
    signal_type = 'df_f'
    use_default = True
    debug_mode = True


run_demo(ExamplePolars, token=TOKEN, clean_cached=False)

In [ ]:
# Loop through all cells with direction preference
class ExamplePolars(VisualPolarOptions):
    exp_date = '210315'
    animal_id = 'YW006'
    plane_index = 0
    signal_type = 'df_f'
    use_default = True


# Loop through all cells with SFTF preference
class ExampleSFTFPref(VisualSFTFPrefOptions):
    exp_date = '210315'
    animal_id = 'YW006'
    plane_index = 0
    signal_type = 'df_f'
    use_default = True
    batch_type = None


# Plot with overall fraction
class SummarySFTFPref(VisualSFTFPrefOptions):
    exp_date = '210315'
    animal_id = 'YW006'
    plane_index = 0
    summary_type = 'fraction'
    used_session = 'light'
    debug_mode = True
    reuse_output = True
    db_debug_mode = True  # non populate .db since need gspread credential


run_demo(ExamplePolars, token=TOKEN, clean_cached=False)
run_demo(ExampleSFTFPref, token=TOKEN, clean_cached=False)
run_demo(SummarySFTFPref, token=TOKEN, clean_cached=False)

## Batch-level Demo
- Analysis that used for multiple recording session in multiple animal
- Use both **sqlite database** and **parquet** dataframe

### SQLite3 Dataframe schema

![Schema](https://github.com/ytsimon2004/rscvp/blob/sql-dev/docs/source/_static/rscvp_sql_schema.png?raw=true)

### Token Request for SQLite database


In [ ]:
# contact author since paper is not published yet
DATABASE_TOKEN = ...
os.environ['RSCVP_DATABASE_PATH'] = database_file = '/content/rscvp/res/database/rscvp.db'
download_database(DATABASE_TOKEN)

### Locations of imaged fields of view
- Used in `Figure 1C`

In [ ]:
# see the field of view table in the database
df = pl.read_database(
    query="SELECT * FROM FieldOfViewDB WHERE usage = 'base'",
    connection=sqlite3.connect(database_file)
)

df


In [ ]:
class ExampleFOV(FieldOfViewOptions):
    database = True
    ibl_res = 25  # save RAM


run_demo(ExampleFOV, token=TOKEN, clean_cached=False)

### Fraction of classified neurons
- Used in `Fig.2A`, `Fig.3D`,`Fig.4D`, `Fig.S2C`, `Fig.S3B–S3C`, `Fig.S4H`

In [ ]:
# see the classified table in the database
df = pl.read_database(
    query='SELECT * FROM BaseClassDB',
    connection=sqlite3.connect(database_file)
)

df

In [ ]:
class ExampleFraction(BaseClassStat):
    header = 'spatial_frac'
    load_source = 'db'


run_demo(ExampleFraction, token=TOKEN, clean_cached=False)

### Parquet dataframe
- .parquet provide for aggregated dataset across sessions (number of neurons x metrics)

### Dorsal Cortex map
- Plot 2D histogram in dorsal cortex view
- Used in `Fig.2C`, `Fig.4D`, `Fig.5K`, `Fig.S3F`


In [ ]:
class ExampleTopoMetric(TopoMetricParQ):
    header = 'si'
    sheet_name = 'spatial_parq'
    scaled = True
    ibl_res = 50


run_demo(ExampleTopoMetric, token=TOKEN, clean_cached=False)

### Population (animal) metric analysis
- show population cumulative + animal-based averaged in metric
- required database for some information (pairwise)

In [ ]:
class ExampleNeuronStat(ValueParQ):
    header = 'si'
    test_type = 'kstest'
    sheet_name = 'spatial_parq'
    plot_type = 'cumulative'
    debug_mode = True


class ExampleAnimalStat(ValueParQ):
    header = 'si'
    test_type = 'kstest'
    sheet_name = 'spatial_parq'
    animal_based_comp = True
    db_table = 'BaseClassDB'  # for join pairwise information
    debug_mode = True


run_demo(ExampleNeuronStat, token=TOKEN, clean_cached=False)
run_demo(ExampleAnimalStat, token=TOKEN, clean_cached=False)